# Molecular Quantum Mechanics (CB2070)
## Computer lab 5: Two-particle densities and electron correlation
---
Name:

Date:

---

In [ ]:
import veloxchem as vlx

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.set_printoptions(precision=4, suppress=True, linewidth=132) # printout format of NumPy arrays
au_to_nm = 0.0529177 # length conversion factor
au_to_ev = 27.2114 # energy conversion factor

### Molecule specification

In [ ]:
mol_str = """
H     0.000000    0.000000   -0.370500
H     0.000000    0.000000    0.370500
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')

### SCF optimization

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
basis = vlx.MolecularBasis.read(molecule, 'sto-3g')
scf_drv.compute(molecule, basis)

### 1. Cost-gain analysis of the bond formation

From the virial theorem we have that
$$2\left<T\right> = n\left<V\right>$$
where $n$ is the exponent of $r$ in the expression for the potential operator $\hat{V}$.

1. What is $n$ in our case?
2. What is the expression for the total average energy?
3. Use this expression and the virial theorem above to get the electron kinetic and potential energies for the isolated H atom. Use the value of `h_h` provided in the code below.

In [ ]:
nocc = molecule.number_of_electrons() // 2
norb = molecule.number_of_atoms()
print('Number of occupied MOs:', nocc)

C = scf_drv.scf_tensors['C_alpha']

# electron kinetic energy
# kin_drv = vlx.KineticEnergyIntegralsDriver()
kin_drv = vlx.KineticEnergyDriver()
T_ao = kin_drv.compute(molecule, basis).to_numpy()
T_mat = np.einsum('ai, ab, bj', C, T_ao, C)
T_e = np.einsum('ii',T_mat[:nocc,:nocc])

# electron-nuclear attraction
# npot_drv = vlx.NuclearPotentialIntegralsDriver()
npot_drv = vlx.NuclearPotentialDriver()
V_ao = -npot_drv.compute(molecule, basis).to_numpy()
V_mat = np.einsum('ai, ab, bj', C, V_ao, C)
V_en = np.einsum('ii',V_mat[:nocc,:nocc])

# core Hamiltonian (one-electron)
h_h2 = T_e + V_en # for the H2 molecule
h_h = 0.5*T_e + 0.25*V_en # for a single H atom

# nuclear-nuclear repulsion
V_nn = molecule.nuclear_repulsion_energy()

# electron-electron repulsion
# eri_drv = vlx.ElectronRepulsionIntegralsDriver()
# eri_ao = np.zeros((norb, norb, norb, norb))
# eri_drv.compute_in_memory(molecule, basis, eri_ao)
fock_drv = vlx.FockDriver()
eri_ao = fock_drv.compute_eri(molecule, basis)
g_mat = np.einsum('abcd,ai,bj,ck,dl', eri_ao, C, C, C, C)
g = 2*np.einsum('iijj',g_mat[:nocc,:nocc,:nocc,:nocc]) - np.einsum('ijji',g_mat[:nocc,:nocc,:nocc,:nocc])

In [ ]:
print('Estimated energy of an isolated H atom: {:3.4f}'.format(h_h))
print('One-electron part of the H2 energy: {:3.4f}'.format(h_h2))
print('Two-electron part of the H2 energy: {:3.4f}'.format(g))
print('Nuclear-nuclear repulsion energy: {:3.4f}'.format(V_nn))

### 2. Explain the (lack of) oupling between blocks in the CISD Hamiltonian

Write your thoughts here...

### 3. Determine the CID state

With $|\Psi \rangle$ being a Slater determinant and $\hat{\Omega}$ a two-electron operator

$$
\hat{\Omega} = \sum_{j>i}^N \hat{\omega}(i,j)
$$

the following relations hold for the corresponding integrals

\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
    \frac{1}{2} \sum_{i,j}^N
    \left[\rule{0pt}{12pt}
    \langle ij| \hat{\omega} | ij \rangle - \langle ij|  \hat{\omega} |ji \rangle
    \right]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{i}^{s} \rangle &=
    \sum_{j}^N
    \left[\rule{0pt}{12pt}
    \langle ij|  \hat{\omega} |sj \rangle - \langle ij|  \hat{\omega} |js \rangle
    \right]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &=
     \langle ij|  \hat{\omega} |st \rangle - \langle ij|  \hat{\omega} |ts \rangle
\end{align*}

Complete the formula
$$\mathbf{H}^{CID} = $$

In [ ]:
# define the CID Hamiltonian using the one- and two-electron integrals determined in (1) above
H_CID = np.zeros((2,2))

H_CID[0,0] = 
H_CID[1,1] = 
H_CID[0,1] = 
H_CID[1,0] = 

print('CID Hamiltonian =\n', H_CID)

# diagonalize the CID Hamiltonian

### 4.  One-particle density

The CID state equals
$$
| \Psi \rangle =  c_0 | \Psi_\mathrm{HF} \rangle + c_1 | \Psi_{u\bar{u}} \rangle
$$

We have
$$
n(\mathbf{r}) =
\langle \Psi | \hat{n}(\mathbf{r}) | \Psi \rangle
$$
with
$$
\hat{n}(\mathbf{r}) = 
\sum_{i=1}^N
\delta(\mathbf{r} - \mathbf{r}_i)
$$

The general relations for matrix elements of a one-electron operator can now be used
\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
   \sum_{i=1}^N
    \langle i| \hat{\omega} |i \rangle
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &= 0
\end{align*}


#### Derive the formula for the one-particle density expressed in terms of spatial orbitals
Complete the equations
$$n^{HF}(\mathbf{r}) = $$

$$n^{CID}(\mathbf{r}) = $$


In [ ]:
# visualization driver
vis_drv = ...

In [ ]:
# get the MOs along the z axis
mol_orbs = scf_results['C_alpha']

n = 100
coords = np.zeros((n,3))
coords[:,2] = np.linspace(-3, 3, n, endpoint=True)

sigma_g = np.array(vis_drv.get_mo(coords, molecule, basis, mol_orbs, 0, 'alpha'))
sigma_u = np.array(vis_drv.get_mo(coords, molecule, basis, mol_orbs, 1, 'alpha'))

# implement your formulas
n1_HF = ...
n1_CID = ...

In [ ]:
fig = plt.figure(figsize = (8,6))

plt.plot(coords[:,2], n1_HF)
plt.plot(coords[:,2], n1_CID)

plt.show()

#### Discuss how the one-particle densities compare between HF and CID

Write your thoughts here...

### 5. Two-particle density
The CID state equals
$$
| \Psi \rangle =  c_0 | \Psi_\mathrm{HF} \rangle + c_1 | \Psi_{g\bar{g}}^{u\bar{u}} \rangle
$$

We have
$$
n(\mathbf{r}, \mathbf{r}') =
\langle \Psi | \hat{n}(\mathbf{r}, \mathbf{r}') | \Psi \rangle
$$
with
$$
\hat{n}(\mathbf{r}, \mathbf{r}') = 
\sum_{j>i}^N \left[
\delta(\mathbf{r} - \mathbf{r}_i) \delta(\mathbf{r}' - \mathbf{r}_j)
+
\delta(\mathbf{r} - \mathbf{r}_j) \delta(\mathbf{r}' - \mathbf{r}_i)
\right]
$$

The general relations for matrix elements of a two-electron operator can now be used
\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
    \frac{1}{2} \sum_{i,j}^N
    \Big[
    \langle ij| \hat{\omega} |ij \rangle - \langle ij | \hat{\omega} | ji \rangle
    \Big]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &=
     \langle ij| \hat{\omega} |st \rangle - \langle ij| \hat{\omega} |ts \rangle
\end{align*}


#### Derive the resulting formula for the two-particle density in terms of spatial orbitals
Complete the equations

$$n^{HF}(\mathbf{r}, \mathbf{r}') =$$

$$n^{CID}(\mathbf{r}, \mathbf{r}') =$$


In [ ]:
# electron 1 at the position of the hydrogen nucleus
# electron 2 anywhere on the internuclear axis

Hz = -0.370500
h1 = [[0, 0, Hz]]

sigma_g_at_h1 = vis_drv.get_mo(h1, molecule, basis, mol_orbs, 0, 'alpha')[0]
sigma_u_at_h1 = vis_drv.get_mo(h1, molecule, basis, mol_orbs, 1, 'alpha')[0]

# implement your formulas
n12_HF = ...
n12_CID = ...

In [ ]:
fig = plt.figure(figsize = (8,6))

plt.plot(coords[:,2], n12_HF)
plt.plot(coords[:,2], n12_CID)

plt.show()

#### Discuss how the two-particle densities compare between HF and CID

Write your thoughts here...

# THE END